# Testing the user defined function

In [1]:
import sys, os
sys.path.insert(0, os.path.abspath('..'))
sys.path.insert(0, os.path.abspath('..')+'/classifiers')

In [2]:
from tuner import Tuner

In [3]:
from scipy.stats import uniform

param_dict = {"n_neighbors": range(1, 50),
              'algorithm' : ['auto', 'ball_tree', 'kd_tree', 'brute']
             }

# userObjective

In [11]:
from CeleryTasks import run_clf_celery

clf_name = 'KNeighborsClassifier'
dataset_name = 'load_breast_cancer'



def objectiveKNNCelery(args_list):
    global clf_name,dataset_name
    process_queue =[]
    
    for hyper_par in args_list:
        process = run_clf_celery.delay(clf_name, dataset_name, hyper_par)
        process_queue.append(process)
    
    results = []
    for process in process_queue:
        result = process.get(timeout=100)
        results.append(result)
           
    return results

In [12]:
conf_Dict = dict()
conf_Dict['batch_size'] = 5
conf_Dict['num_iteration'] = 5

# Defining Tuner

In [13]:
tuner_user = Tuner(param_dict, objectiveKNNCelery, conf_Dict)

In [14]:
tuner_user.getConf()

{'param_dict': {'n_neighbors': range(1, 50),
  'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute']},
 'userObjective': <function __main__.objectiveKNNCelery(args_list)>,
 'domain_size': 5000,
 'initial_random': 1,
 'num_iteration': 5,
 'objective': 'maximize',
 'batch_size': 5}

In [15]:
results = tuner_user.run()

# Inspect the results

In [ ]:
print('best hyper parameters:',results['best_hyper_parameter'])
print('best objective:',results['best_objective'])

In [ ]:
print('Sample hyper parameters tried:',len(results['hyper_parameters_tried']))
print(results['hyper_parameters_tried'][:2])

In [ ]:
print('Sample objective values',len(results['objective_values']))
print(results['objective_values'][:5])

# Plotting the variation of actual objective values of the tried results

In [ ]:
Size = 201 

from matplotlib import pyplot as plt
fig = plt.figure(figsize=(30,5))
plt.title('Variation of Objective',fontsize=20)
plt.plot(results['objective_values'][:Size],lw=4,label='BL')
plt.xlabel('Configurations', fontsize=25)
plt.ylabel('objective_values',fontsize=25)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.legend(prop={'size': 30})
plt.show()

# Plotting the variation of Max objective values of the tried results

In [ ]:
Size = 201 

import numpy as np

results_obj = np.array(results['objective_values'])

y_max=[]
for i in range(results_obj.shape[0]):
    y_max.append(np.max(results_obj[:i+1]))

from matplotlib import pyplot as plt
fig = plt.figure(figsize=(30,5))
plt.title('Max variation of Objective',fontsize=20)
plt.plot(y_max[:Size],lw=4,label='BL')
plt.xlabel('Configurations', fontsize=25)
plt.ylabel('objective_values',fontsize=25)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.legend(prop={'size': 30})
plt.show()

# See the Result

In [ ]:
import pprint
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(results)